In [15]:
import requests
import pandas as pd
import numpy as np
import time

In [16]:
API_key = '5096feff1c44572580c0389b8fb4b952'

**<font size="5">Get Data from API</font>**

**Function to get movie data sorted by revenue**  

In [17]:
def get_data(keywords, num_pages, api=API_key):
    data = []
    for i in range(1, num_pages):
        query = 'https://api.themoviedb.org/3/discover/movie?api_key=' +  api + keywords +'&page=' + str(i)
        response =  requests.get(query)
        if response.status_code==200: 
            data.append(response.json())
        else:
            return "error"
    return data

**Call Function, Iterate through the Resulting pages, and Collect Necessary Information**

In [19]:
film_data = get_data('&sort_by=revenue.desc', 500)
columns = ['title', 'revenue', 'budget', 'release_date', 'genres', 'popularity', 'original_language', 'production_companies', 'id', 'runtime', 'adult', 'cast', 'crew']
df = pd.DataFrame(columns=columns)

# Check whether keys are in json output
fm_set = columns[:-2]
fc_set = columns[-2:]
start = time.time()
# Iterate through json data and add attributes were interested in to the DataFrame
for page in film_data:
    for film in page['results']:
        film_revenue = requests.get('https://api.themoviedb.org/3/movie/' + str(film['id']) + '?api_key=' + API_key + '&language=en-US').json() 
        film_cast = requests.get('https://api.themoviedb.org/3/movie/' + str(film['id']) +'/credits?api_key='+ API_key+'&language=en-US').json() 
        if set(fm_set).issubset(film_revenue.keys()) and set(fc_set).issubset(film_cast.keys()): # If entries are filled for a given film
            df.loc[len(df)] = [film['title'], film_revenue['revenue'], film_revenue['budget'], 
                   film_revenue['release_date'], film_revenue['genres'], film_revenue['popularity'],
                   film_revenue['original_language'], film_revenue['production_companies'], film_revenue['id'],
                   film_revenue['runtime'], film_revenue['adult'], film_cast['cast'], film_cast['crew']]
        else: # Discard film
            continue
end = time.time()
print('Time to call 500 pages of records: {} seconds'.format(end - start))

Time to call 500 pages of records: 0.00016808509826660156 seconds


**<font size="5">Format Necessary Columns</font>**

**Row-Wise Function to Pull Data from the JSON Loads in DataFrame**  

In [ ]:
def read_json(row, group_num):
    list_of_names = []
    if group_num:
        for item in row:
            list_of_names.append('{} | {}'.format(item['name'], item['popularity']))
    else:
        for item in row:
            list_of_names.append(item['name'])
    return ', '.join(list_of_names)

**Call Function on Necessary Columns in DataFrame**

In [ ]:
# Group 1 refers to crew/cast (we want name : popularity)
df['crew'] = df['crew'].apply(lambda d: read_json(d, 1))
df['cast'] = df['cast'].apply(lambda d: read_json(d, 1))

# Group 0 refers to genres/production_companies (we just want name)
df['genres'] = df['genres'].apply(lambda d: read_json(d, 0))
df['production_companies'] = df['production_companies'].apply(lambda d: read_json(d, 0))

**Save to File**

In [ ]:
df.to_excel('../Data/TMDB_dataset.xlsx', index=False)
# df.to_csv('../Data/TMDB_dataset.csv', index=False)